In [ ]:
import datetime
import geopandas
import joblib
import pathlib
import urllib.parse
import xyzservices

In [ ]:
CACHE_DIRECTORY = pathlib.Path() / ".cache"
CACHE_TIME = datetime.timedelta(weeks=3)

CACHE_DIRECTORY.mkdir(exist_ok=True)
memory = joblib.Memory(CACHE_DIRECTORY, verbose=0)
memory.reduce_size(age_limit=CACHE_TIME)

In [ ]:
CRS = "EPSG:31256"

WFS_BASE_URL = (
    "https://data.wien.gv.at/daten/geo"
    "?service=WFS"
    "&version=2.0.0"
    "&request=GetFeature"
    f"&srsName={CRS}"
    "&typeNames={:s}"
)

PARKFLAECHEN_AUF_GEHSTEIG_WFS_URL = (
    WFS_BASE_URL.format("ogdwien:SISBELAGOGD")
    + "&"
    + urllib.parse.urlencode({"cql_filter": "TYPE_TXT='Parkfläche Gehsteig'"})
)
GEHSTEIG_WFS_URL = (
    WFS_BASE_URL.format("ogdwien:SISBELAGOGD")
    + "&"
    + urllib.parse.urlencode({"cql_filter": "TYPE_TXT='Gehsteig'"})
)
ORTHOFOTO_TILE_URL = (
    "https://mapsneu.wien.gv.at/wmts/lb/farbe/google3857/{z}/{y}/{x}.jpeg"
)
xyzservices.providers.WienOGD = xyzservices.Bunch(
    Orthofoto=xyzservices.TileProvider(
        name="Orthofoto Wien",
        url=ORTHOFOTO_TILE_URL,
        attribution="Datenquelle: Stadt Wien – data.wien.gv.at",
    )
)

In [ ]:
MAX_DISTANCE = 0.5

In [ ]:
GEHSTEIGE_FILL_COLOUR = "#83ba7e"
GEHSTEIGE_STROKE_COLOUR = "#83ba7e"
PARKPLAETZE_FILL_COLOUR = "#73a0dc"
PARKPLAETZE_STROKE_COLOUR = "#73a0dc"

In [ ]:
@memory.cache
def cached_geopandas_read_file(*args, **kwargs):
    return geopandas.read_file(*args, **kwargs)

In [ ]:
parkflaechen = cached_geopandas_read_file(PARKFLAECHEN_AUF_GEHSTEIG_WFS_URL)
parkflaechen.crs = CRS
parkflaechen

In [ ]:
gehsteig = cached_geopandas_read_file(GEHSTEIG_WFS_URL)
gehsteig.crs = CRS
gehsteig

In [ ]:
gehsteig_neben_parkflaeche = gehsteig.sjoin_nearest(
    parkflaechen,
    max_distance=MAX_DISTANCE,
    lsuffix="",
    rsuffix="parkflaeche",
    distance_col="Abstand Gehsteig/Parkfläche",
)
gehsteig_neben_parkflaeche

In [ ]:
gehsteig_neben_parkflaeche["Abstand Gehsteig/Parkfläche"].describe()

In [ ]:
x = gehsteig_neben_parkflaeche.dissolve().explode()  # .area.describe()

In [ ]:
# uebersichtskarte = gehsteige.explore(
uebersichtskarte = gehsteig_neben_parkflaeche.explore(
    style_kwds={
        "fillColor": GEHSTEIGE_FILL_COLOUR,
        "fillOpacity": 0.9,
        "color": GEHSTEIGE_STROKE_COLOUR,
        "weight": 0.1,
    },
    map_kwds={
        "center": {"lat": 48.2515, "lng": 16.4549},
    },
    zoom_start=18,
    max_zoom=32,
    tiles=xyzservices.providers.WienOGD.Orthofoto,
)
uebersichtskarte = parkflaechen.explore(
    m=uebersichtskarte,
    style_kwds={
        "fillColor": PARKPLAETZE_FILL_COLOUR,
        "fillOpacity": 0.9,
        "color": PARKPLAETZE_STROKE_COLOUR,
        "weight": 0.1,
    },
)
uebersichtskarte